In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pyNUISANCE as pn
import pyProSelecta as pps

Welcome to JupyROOT 6.30/04


In [2]:
%env NUISANCE3_ROOT="/Users/juliatenavidal/Desktop/Postdoc/Tensions/nuisance3-src/build"

env: NUISANCE3_ROOT="/Users/juliatenavidal/Desktop/Postdoc/Tensions/nuisance3-src/build"


In [ ]:
#For electrons:
achilles_ep_500 = pn.EventSource("Achilles/electron_500.hepmc")
achilles_ep_1000 = pn.EventSource("Achilles/electron_1000.hepmc")
achilles_ep_2000 = pn.EventSource("Achilles/electron_2000.hepmc")

genie_ep_500 = pn.EventSource("GENIE/e_on_1000010010_500MeV_0.hepmc3")
genie_ep_1000 = pn.EventSource("GENIE/e_on_1000010010_1000MeV_0.hepmc3")
genie_ep_2000 = pn.EventSource("GENIE/e_on_1000010010_2000MeV_0.hepmc3")

gibuu_ep_500 = pn.EventSource("Gibuu/EventOutput_e500MeV.Pert.hepmc3")

# For neutrinos:
achilles_nub_500 = pn.EventSource("Achilles/electron_500.hepmc")
achilles_nub_1000 = pn.EventSource("Achilles/electron_1000.hepmc")
achilles_nub_2000 = pn.EventSource("Achilles/electron_2000.hepmc")

#genie_nub_500 = pn.EventSource("GENIE/CCQE_nuebar_G18_10a_02_11a_0p5GeV.ghep.root")
#genie_nub_1000 = pn.EventSource("GENIE/CCQE_nuebar_G18_10a_02_11a_1GeV.ghep.root")
#genie_nub_2000 = pn.EventSource("GENIE/CCQE_nuebar_G18_10a_02_11a_2GeV.ghep.root")

neut_nub_500 = pn.EventSource("NEUT/NEUT_nieves_1.05_nueb_CH2_0.5GeV.nuis.root.GenericVectors.root")
neut_nub_500 = pn.EventSource("NEUT/NEUT_nieves_1.05_nueb_CH2_1GeV.nuis.root.GenericVectors.root")
neut_nub_500 = pn.EventSource("NEUT/NEUT_nieves_1.05_nueb_CH2_2GeV.nuis.root.GenericVectors.root")

In [ ]:
def calc_q2(ev):
    val = [pps.kMissingDatum]
    
    if pps.event.has_beam_part(ev, 11):
        mom0 = np.array([pps.event.beam_part(ev, 11).momentum().e() / pps.unit.GeV,
                         pps.event.beam_part(ev, 11).momentum().px() / pps.unit.GeV,
                         pps.event.beam_part(ev, 11).momentum().py() / pps.unit.GeV,
                         pps.event.beam_part(ev, 11).momentum().pz() / pps.unit.GeV])
        
    if pps.event.has_out_part(ev, 11):
        mom1 = np.array([pps.event.hm_out_part(ev, 11).momentum().e() / pps.unit.GeV,
                         pps.event.hm_out_part(ev, 11).momentum().px() / pps.unit.GeV,
                         pps.event.hm_out_part(ev, 11).momentum().py() / pps.unit.GeV,
                         pps.event.hm_out_part(ev, 11).momentum().pz() / pps.unit.GeV])

    val = (mom0 - mom1)
    val = -(val[0]**2-val[1]**2-val[2]**2-val[3]**2)
    
    return [val]

In [ ]:
pb_to_nb = 1/1000

def compare(achilles, genie, title=None, qmin=0, qmax=2, nbins=100, name="test"):
    achilles_ev_frame = pn.EventFrameGen(achilles).add_columns(["Q2"], calc_q2).all()
    genie_ev_frame = pn.EventFrameGen(genie).add_columns(["Q2"], calc_q2).all()

    achilles_hist = pn.HistFrame(pn.Binning.lin_spaceND([[qmin, qmax, nbins]], [r'$Q^2$ [GeV]']))
    achilles_hist.fill_from_EventFrame(achilles_ev_frame, ["Q2"])
    achilles_hist = achilles_hist.finalise().scale(achilles_ev_frame.norm_info().fatx_per_sumweights() * pb_to_nb)
    achilles_hist.mpl().hist_all(histtype="step", labels=['Achilles'])

    genie_hist = pn.HistFrame(pn.Binning.lin_spaceND([[qmin, qmax, nbins]], [r'$Q^2$ [GeV]']))
    genie_hist.fill_from_EventFrame(genie_ev_frame, ["Q2"])
    genie_hist = genie_hist.finalise()
    print(genie_ev_frame.norm_info().fatx_per_sumweights())
    #.scale(genie_ev_frame.norm_info().fatx_per_sumweights() * pb_to_nb)
    genie_hist.mpl().hist_all(histtype="step", labels=['Genie'])

    if title:
        plt.title(title)

    plt.legend()
    plt.savefig(f"{name}.pdf", bbox_inches="tight")
    plt.show()

In [ ]:
compare(achilles_ep_500, genie_ep_500, title="500 MeV", qmin=0, qmax=2, nbins=200, name="electron_500MeV")

In [ ]:
print(achilles_ev_frame.norm_info().fatx, achilles_ev_frame.norm_info().sumweights, achilles_ev_frame.norm_info().nevents)